In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.utils.class_weight import compute_class_weight

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Diabetes\\data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 300, 300  # EfficientNetB3 prefers higher resolution
BATCH_SIZE = 64
EPOCHS = 30

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 📏 Class Weight Calculation
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# 🧠 Load EfficientNetB3 Base Model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False

# 🏗️ Custom Classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_effnetb3_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Train Phase 1: Feature Extraction
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Unfreeze for Fine-tuning
base_model.trainable = True

# Recompile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(), Precision(), Recall()]
)

# 🚀 Train Phase 2: Fine-Tuning
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Final Evaluation
loss, accuracy, auc, precision, recall = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Test Accuracy: {accuracy * 100:.2f}%")
print(f"📈 AUC: {auc:.4f} | 🎯 Precision: {precision:.4f} | 🔍 Recall: {recall:.4f}")

# 💾 Save Final Model
model.save("final_effnetb3_model.h5")
print("✅ Model training completed with high accuracy!")


Found 112 images belonging to 2 classes.
Found 24 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5078 - auc: 0.4504 - loss: 0.7498 - precision: 0.4848 - recall: 0.2411  
Epoch 1: val_accuracy improved from -inf to 0.79167, saving model to best_effnetb3_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 48s 15s/step - accuracy: 0.5260 - auc: 0.4771 - loss: 0.7387 - precision: 0.5253 - recall: 0.2798 - val_accuracy: 0.7917 - val_auc: 0.8993 - val_loss: 0.5759 - val_precision: 0.7692 - val_recall: 0.8333 - learning_rate: 3.0000e-04
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6786 - auc: 0.7596 - loss: 0.5981 - precision: 0.6450 - recall: 0.8403
Epoch 2: val_accuracy improved from 0.79167 to 0.83333, saving model to best_effnetb3_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 9s/step - accuracy: 0.6756 - auc: 0.7483 - loss: 0.6054 - precision: 0.6378 - recall: 0.8459 - val_accuracy: 0.8333 - val_auc: 0.8958 - val_loss: 0.5055 - val_precision: 0.8333 - val_recall: 0.8333 - learning_rate: 3.0000e-04
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8616 - auc: 0.8841 - loss: 0.4631 - precision: 0.8093 - recall: 0.9532
Epoch 3: val_accuracy did not improve from 0.83333
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 9s/step - accuracy: 0.8571 - auc: 0.8803 - loss: 0.4638 - precision: 0.8032 - recall: 0.9510 - val_accuracy: 0.7500 - val_auc: 0.8958 - val_loss: 0.4844 - val_precision: 0.8000 - val_recall: 0.6667 - learning_rate: 3.0000e-04
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8467 - auc: 0.8861 - loss: 0.4423 - precision: 0.8229 - recall: 0.8571
Epoch 4: val_accuracy did not improve from 0.83333
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 9s/step - accuracy: 0.8442 - auc: 0.8819 - loss: 0.4444 - precision: 0.8245 - recall: 0.8571 - 